## Create GO slims for BaRTv2.18 annotation

In [26]:
!source activate owltools

In [29]:
import os
os.chdir("/cluster/db/mecoulter/CEMartinez_RNAseq-209915706/results/GO/")

* Input files:

In [30]:
gene_annotation = "/cluster/db/mecoulter/BaRT2v18/BaRT_2_18_annotation_genes.txt"
GO_annotation = "BaRTv2.18_genes_go_ids.txt"

* First create list of genes with GO terms:

In [5]:
import pandas as pd
annotation = pd.read_csv(gene_annotation, delimiter="\t",index_col=0)
annotation.head()

,Chromosome,Start,End,Strand,Sources,End support,Pannzer annotation,GO IDs,GO terms,Coding potentiality
BaRTv2 gene,,,,,,,,,,
BaRT2v18chr1HG000020,chr1H,72060,73355,+,Illumina;HC Iso-Seq,Illumina;Enriched TSS and TES,"50S ribosomal protein L28, chloroplastic",GO:0097159;GO:0005783;GO:0003723;GO:0012505;GO...,binding;organelle envelope;intracellular membr...,Unproductive;Coding
BaRT2v18chr1HG000030,chr1H,72100,73281,-,Illumina,Illumina,NaN,NaN,NaN,Non_Coding
BaRT2v18chr1HG000040,chr1H,102316,103908,+,HC Iso-Seq;HC Iso-Seq,Enriched TSS and TES;Enriched TSS and TES,NaN,GO:0097159;GO:0005488;GO:0003676;GO:1901363,nucleic acid binding;organic cyclic compound b...,Coding;Coding
BaRT2v18chr1HG000050,chr1H,102332,108065,-,Illumina;Illumina;HC Iso-Seq;HC Iso-Seq;HC Iso...,Illumina;Illumina;Enriched TSS and TES;Enriche...,Peroxin-14,GO:0072662;GO:0044743;GO:0016558;GO:0006996;GO...,protein binding;signaling receptor binding;bin...,Coding;Coding;Unproductive;Coding;Unproductive...
BaRT2v18chr1HG000060,chr1H,110052,113271,-,Illumina;HC Iso-Seq;HC Iso-Seq;HC Iso-Seq;HC I...,Illumina;Enriched TSS and TES;Enriched TSS and...,RING-finger ubiquitin ligase;E3 ubiquitin-prot...,GO:0071310;GO:0009756;GO:0006508;GO:0031410;GO...,intracellular membrane-bounded organelle;estab...,Coding;Unproductive;Unproductive;Unproductive;...


In [23]:
with open(GO_annotation,"w") as out:
    for gene in annotation.index:
        try:
            for GO in annotation.at[gene,"GO IDs"].split(";"):
                out.write(f"{gene}\t{GO}\n")
        except AttributeError:
            pass

* Create GAF file:

In [24]:

output = "BaRTv2.18_GO.gaf"



with open(output, "w") as out:
        out.write("!gaf-version: 2.0\n")
        for line in open(GO_annotation):
                gene, GO_id = line.strip().split("\t")
                out.write(f"WB\t{gene}\t{gene}\toptional\t{GO_id}\tpubmed\tunknown\toptional\tunknown\t\toptional\toptional\tprotein\tunknown\t{gene}\tWB\toptional\toptional\n")

In [31]:
!echo $PWD

/cluster/db/mecoulter/CEMartinez_RNAseq-209915706/results/GO


In [15]:
#<grep "id: GO:" goslim_plant.obo | grep -v alt | cut -d" " -f2 > goslim_terms.txt> run in terminal

* Create GO slims. INstructions from:
* go-basic.obo from:
* goslim_terms.txt are plant GO slim terms

In [ ]:
!/cluster/db/mecoulter/CEMartinez_RNAseq-209915706/results/GO/owltools/OWLTools-Runner/target/owltools go-basic.obo --gaf BaRTv2.18_GO.gaf --map2slim --idfile goslim_terms.txt --write-gaf BaRTv2.18_slim.txt

In [33]:
input1 = "goslim_plant.obo"
outfile = "goslim_ids_descriptions.txt"

with open(outfile,"w") as out:
	out.write("term_id\tdescription\n")
	is_GO = False
	for line in open(input1):
		if line.startswith("[Term]"):#Line [Term] must appear before id
			is_GO = True
		if line.startswith("id:") and is_GO:
			ID = line.strip().replace("id: ","")
			out.write(ID + "\t")
		elif line.startswith("name:") and is_GO:
			name = line.strip().replace("name: ","")
			out.write(name + "\n")
			is_GO = False
		else:
			continue

In [34]:
#Now create other input file for GO_slim analysis:
#gene_id and term_id
input1 = "BaRTv2.18_slim.txt"
outfile = "BaRTv2.18_gene_slim.txt"

gene_terms = {}
allGOslims = set()
for line in open(input1):
	if line.startswith("!"):
		continue
	fields = line.split("\t")
	gene = fields[1]
	term = fields[4]
	allGOslims.add(term)
	gene_terms.setdefault(gene,set()).add(term)

with open(outfile,"w") as out:
	out.write("gene_id\tterm_id\n")
	for gene, term_set in gene_terms.items():
		for term in term_set:
			out.write(gene + "\t" + term + "\n")